<a href="https://colab.research.google.com/github/Prasadchaskar/Flowers-Classification/blob/main/flowers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
from pprint import pprint

In [24]:
BATCH_SIZE = 20
IMAGE_SIZE = 180
CHANNELS=3
EPOCHS=10

In [25]:
# import zipfile
# !unzip /content/drive/MyDrive/flowers.zip

In [26]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/17_flowers",
    seed=123,
    shuffle=True,
    image_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE
) 

Found 1360 files belonging to 2 classes.


In [27]:
class_names = dataset.class_names

In [28]:
def get_dataset_partitions_tf(ds, train_split=0.8,test_split=0.2, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split) == 1
    
    ds_size = len(ds)
    
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)
    
    train_size = int(train_split * ds_size)
    val_size = int(test_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds,test_ds

In [29]:
train_data,test_data = get_dataset_partitions_tf(dataset)

In [30]:
train_ds = train_data.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_data.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_data.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [31]:
resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
  layers.experimental.preprocessing.Rescaling(1./255),
])

In [32]:
data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomRotation(0.2),
])

In [33]:

train_ds = train_ds.map(
    lambda x, y: (data_augmentation(x, training=True), y)
).prefetch(buffer_size=tf.data.AUTOTUNE)

In [34]:
input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes = 17
model = models.Sequential([
    resize_and_rescale,
    layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    # layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    # layers.MaxPooling2D((2, 2)),
    # layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),
])

model.build(input_shape=input_shape)

In [35]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [36]:
history = model.fit(
        train_ds,
        batch_size=BATCH_SIZE,
        validation_data=val_ds,
        verbose=1, 
        epochs=13,
    )

Epoch 1/13
54/54 [==============================] - 9s 116ms/step - loss: 3.5079 - accuracy: 0.7907 - val_loss: 0.4775 - val_accuracy: 0.8438
Epoch 2/13
54/54 [==============================] - 4s 77ms/step - loss: 0.3705 - accuracy: 0.8833 - val_loss: 0.4992 - val_accuracy: 0.8438
Epoch 3/13
54/54 [==============================] - 4s 77ms/step - loss: 0.3588 - accuracy: 0.8861 - val_loss: 0.5603 - val_accuracy: 0.8438
Epoch 4/13
54/54 [==============================] - 4s 76ms/step - loss: 0.3657 - accuracy: 0.8833 - val_loss: 0.5908 - val_accuracy: 0.8438
Epoch 5/13
54/54 [==============================] - 4s 75ms/step - loss: 0.3514 - accuracy: 0.8843 - val_loss: 0.4494 - val_accuracy: 0.8516
Epoch 6/13
54/54 [==============================] - 4s 75ms/step - loss: 0.3533 - accuracy: 0.8824 - val_loss: 0.4769 - val_accuracy: 0.8438
Epoch 7/13
54/54 [==============================] - 4s 77ms/step - loss: 0.3525 - accuracy: 0.8833 - val_loss: 0.5252 - val_accuracy: 0.8438
Epoch 8/13
5